In [1]:
pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver  # Automate web browser control
from selenium.webdriver.edge.options import Options  # Options for Edge browser
from selenium.webdriver.common.by import By  # Search for elements on the web page
from selenium.webdriver.support.ui import WebDriverWait  # Wait for a certain condition before the next operation
from selenium.webdriver.support import expected_conditions as EC  # Expected conditions in Selenium
import pandas as pd  
from datetime import datetime  
import time  # Library to create delay during execution

#### GOLD

In [3]:
url = 'https://finance.yahoo.com/quote/GC%3DF/history?p=GC%3DF'
start_date_input = datetime(2023,11,3)
end_date_input = datetime(2023,12,20)

In [4]:

# Step 1: set def with parameters
def get_data(url, start_date_input, end_date_input):
    
    options = webdriver.EdgeOptions()  # create an EdgeOptions object
    options.add_experimental_option('detach', True)  # add the 'detach' option - website runs in detached mode
    driver = webdriver.Edge(options=options)  # create a webdrive object using the Edge browser and pass it the options object
    driver.get(url)
    print('Finish step 1!')
    
    # Step 2: set range date 
    def set_date_and_apply(start_date, end_date):
        time.sleep(5)
        date_picker = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div[1]')
        date_picker.click()

        # Convert datetime objects to strings
        start_date_str = start_date.strftime('%d-%m-%Y')
        end_date_str = end_date.strftime('%d-%m-%Y')

        start_input = driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/div[1]/input')
        start_input.clear()
        start_input.send_keys(start_date_str)

        end_input = driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/div[2]/input')
        end_input.clear()
        end_input.send_keys(end_date_str)

        apply_button = driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/div[3]/button[1]')
        apply_button.click()

        apply_button = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button')
        apply_button.click()

    set_date_and_apply(start_date_input, end_date_input)
    print('Finish step 2!')
    
    # Step3: Scroll down to download all data, (if you haven't scrolled down yet, the data will only load up to 100 lines)
    
    time.sleep(5)  # Wait for initial content to load
    previous_length = 0

    while True:
        # Scroll down to the bottom of the table
        bottom = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tfoot/tr')
        driver.execute_script("arguments[0].scrollIntoView(true);", bottom)
        time.sleep(5)  # Wait for the webpage to update after each scroll

        # Check if the number of rows has changed
        current_length = len(driver.find_elements(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr'))
        if current_length == previous_length:
            break  # If no more data is loaded, exit the loop
        else:
            previous_length = current_length  # Update the previous length to the current length

    # Retrieve all rows from the table
    rows = driver.find_elements(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr')
    print('Finish step 3!')
    
    # Step 4: crawl rows, (avoid index out of the range, when collect for col)
    data = []   # list save
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')     
        # find_elements, find all cells (td tags in HTML). find_elements(By.TAG_NAME, ...), find_elements(By.XPATH, ...)


        if len(cells) >= 5:

            date_text = cells[0].text
            open_text = cells[1].text
            high_text = cells[2].text
            low_text = cells[3].text
            close_text = cells[4].text    # close ở cột cells[4]

            try:
                    f = lambda x: float(x.replace(',', ''))
                    open_, high_, low_, close_ = f(open_text), f(high_text), f(low_text), f(close_text)


                    numerical_date = datetime.strptime(date_text, '%b %d, %Y').strftime('%d/%m/%Y')
                    # datetime - Convert STRP string to STRF specific date %Y-%m-%d, or %d-%m-%Y, ... (for example from 'Jan 01, 2020' to ' 2020-01-01').
                    data.append({'Date': numerical_date,
                                 'Open': open_,
                                 'High': high_,
                                 'Low': low_,
                                 'Close': close_})
            except:
                print('Error:', date_text)
    print('Finish step 4!')
    
    df = pd.DataFrame(data)
    
    return df

In [5]:
start = datetime(2023,11,3)
end = datetime(2023,12,20)
link = 'https://finance.yahoo.com/quote/GC%3DF/history?p=GC%3DF'
df = get_data(link, start, end)

Finish step 1!
Finish step 2!
Finish step 3!
Finish step 4!


In [6]:
df

,Date,Open,High,Low,Close
0,19/12/2023,2025.2,2042.7,2025.2,2038.4
1,18/12/2023,2024.3,2032.8,2024.0,2026.3
2,15/12/2023,2032.6,2043.4,2018.3,2021.1
3,14/12/2023,2024.7,2040.1,2024.3,2030.2
4,13/12/2023,1978.5,2024.8,1975.0,1982.3
5,12/12/2023,1984.2,1994.2,1977.8,1977.8
6,11/12/2023,2004.1,2004.2,1977.2,1978.0
7,08/12/2023,2031.7,2033.1,1995.0,1998.3
8,07/12/2023,2029.2,2034.9,2024.0,2029.9
9,06/12/2023,2019.9,2035.8,2019.0,2030.5


In [7]:
df.to_csv('Gold.csv')